# HF 一阶核坐标梯度性质

这两节中，我们会回顾 HF 方法下核坐标梯度有关的性质，这包括 HF 一阶坐标梯度与二阶坐标梯度．HF 一阶坐标梯度可以用于几何结构优化，而二阶坐标梯度则可以给出分子频率信息，进而得到平衡构型下简正振动模式．这些都是分子性质分析中最为常用的功能．

由于核坐标梯度性质的内容相对复杂，这一节我们首先回顾一阶坐标梯度性质；并利用一阶坐标梯度，简单了解借助于 [berny 库](https://jan.hermann.name/pyberny/) 的结构优化程序实现．

坐标梯度与电场梯度都是梯度性质，它们不管从公式或者程序导出上都有许多相似之处；但坐标梯度的很大的不同之处在于，一方面，核坐标梯度下，双电子积分与重叠积分的偏导数将不再是零，梯度公式多少会复杂一些．另一方面，核坐标梯度的被求导对象是原子坐标，若有 $N$ 个原子，那么依照笛卡尔坐标，就有 $3N$ 个被求导对象；这多少会增加代码的复杂性．同时，积分库一般不支持直接对核坐标的求导；尽管不困难，但大多数时候我们需要手动推导对核坐标求导与对电子坐标求导的关系，这也会增加复杂性，且容易产生错误的推导．

In [ ]:
from pyscf import gto, scf, grad, data, lib
import numpy as np
np.set_printoptions(5, linewidth=150, suppress=True)

## 准备工作

### 顶层函数计算 HF 梯度

In [ ]:
mol = gto.Mole()
mol.atom = """
O  1.0  0.0  0.0
H  1.0  1.0  0.0
H  1.0  0.0  1.0
"""
mol.basis = "6-31G"
mol.build()

In [ ]:
scf_eng = scf.RHF(mol)
energy_RHF = scf_eng.kernel()

HF 的核坐标梯度在 PySCF 中可以通过下述代码给出：

In [ ]:
scf_grad = grad.RHF(scf_eng)
grad_RHF = scf_grad.kernel()

### HF 重要中间矩阵

In [ ]:
nao = mol.nao
nmo = scf_eng.mo_energy.shape[0]
nelec = mol.nelectron
nocc = mol.nelec[0]
nvir = nmo - nocc

S = mol.intor('int1e_ovlp_sph')
T = mol.intor('int1e_kin_sph')
Vnuc = mol.intor('int1e_nuc_sph')
eri = mol.intor('int2e_sph')

C = scf_eng.mo_coeff
Co = C[:, :nocc]
Cv = C[:, nocc:]
e = scf_eng.mo_energy
eo = e[:nocc]
ev = e[nocc:]
mo_occ = scf_eng.mo_occ

D = scf_eng.make_rdm1()
F = scf_eng.get_fock()

J = scf_eng.get_j()
K = scf_eng.get_k()

## HF 一阶核坐标梯度公式

### 核坐标梯度公式

现在用较为简化的公式表达核坐标梯度．根据 Yamaguchi (V.1)，如果 $a$ 代表一个原子的坐标方向分量，那么体系电子态能量对 $a$ 的导数为

\begin{equation}
\frac{E_\mathrm{elec}}{\partial a} = 2 h_{ii}^a + 2 (ii|jj)^a - (ij|ij)^a - 2 S_{ii}^a \varepsilon_i
\end{equation}

注意到上面的公式中，没有考虑原子核间排斥能关于 $a$ 的导数对体系能量导数的贡献．事实上，$\partial_a E_\mathrm{nuc}$ 一般不为零；即上面公式并不是分子总能量的一阶梯度．

### 基组与原子核

在进行程序说明前，我们需要先对 PySCF 处理基组与原子核之间关系的方式作了解．在这里，我们使用了 6-31G 的双 $\zeta$ 基组，这意味着对于价层电子，其一层的 CGTO 基组会分成两个 $\zeta$ 函数来描述，且这两个 $\zeta$ 函数分别是 3 个、1 个 GTO 函数以确定的系数线性组合而成．而核层电子则只使用一个 $\zeta$ 函数描述，这个函数以 6 个 GTO 函数以确定系数线性组合而成．

不过既然上面这些构成 $\zeta$ 函数的系数是确定的，那么我们不必要再研究这些系数；而只需要关心核层有 1 个、价层有 2 个 $\zeta$ 函数这个事件即可．对于氧原子，我们说它由 5 种 $\zeta$ 函数构成，分别是核层的 $1s$ 与价层的 $2s, 2p$．而又由于 $2p$ 轨道有取向上的差别，即 $2p_x, 2p_y, 2p_z$，因此总共有 9 中不同取向的 $\zeta$ 函数．

这可以在 PySCF 的函数 `gto.mole.Mole.aoslice_by_atom` 中显示．我们可以看到，对于第一组列表，前两个值的差减表示氧原子的 5 种 $\zeta$ 函数数量；而后两个值的差减表示氧原子的不同取向的 $\zeta$ 函数的数量．同理地，对于氢原子也同样如此：氢原子有价层的 $1s$ 轨道，因此其具有 2 种 $\zeta$ 函数；又由于 $1s$ 轨道不具有取向差别，因此不同取向的 $\zeta$ 函数仍然是两个．

In [ ]:
mol.aoslice_by_atom()

我们可以发现，上述列表的最后一个值就是原子基组数量．事实上，我们用到的系数矩阵 $C_{\mu p}$ 的第一个下标中，如果 $0 \leqslant \mu < 9$，则这些代表了氧原子；而 $9 \leqslant \mu < 11$ 代表第一个氢原子，$11 \leqslant \mu < 13$ 代表第二个氢原子．

## 核哈密顿量导数对梯度的贡献

### 核哈密顿量梯度的表达式

我们先解决梯度公式的第一项．首先回顾一下核哈密顿量 (Core Hamiltonian) 与其梯度的详细结论．我们知道，对于一个电子而言，其核哈密顿量为

\begin{equation}
h_{\mu \nu} = \langle \mu | -\frac 1 2 \nabla^2 + \frac{-Z_M}{| \boldsymbol{r} - \boldsymbol{M} |} | \nu \rangle
\end{equation}

其中，上式的算符 $\nabla^2$ 实际上可以表示为关于电子坐标 $\boldsymbol{r}$ 的函数与算符，$\boldsymbol{r}$ 也并不是作为 Einstein Summation Convention 中的被求和对象；$\mu, \nu$ 不仅与 $\boldsymbol{r}$ 有关，还因为它们是原子轨道基组，因此与其所在原子位置也有关；而 $\boldsymbol{A}$ 则代表原子的坐标位置．我们简记上式中的算符为动能与势能算符为

\begin{align}
\hat t &= - \frac 1 2 \nabla_{\boldsymbol{r}}^2 \\
{\hat v}_M &= \frac{-Z_M}{| \boldsymbol{r} - \boldsymbol{M} |}
\end{align}

那么，如果我们定义被偏导变量 $a = \boldsymbol{A}$ 为原子坐标向量，那么

\begin{equation}
h_{ii}^\boldsymbol{A} = C_{\mu i} C_{\nu i} \nabla_\boldsymbol{A} \langle \mu | \hat t + \hat v_M | \nu \rangle
\end{equation}

以后会记 $A_t$ 为 $\boldsymbol{A}$ 的坐标分量，其中 $t \in \{ x, y, z \}$．那么以核哈密顿为例，可以表示为

\begin{equation}
h_{ii}^{A_t} = C_{\mu i} C_{\nu i} \partial_{A_t} \langle \mu | \hat t + \hat v_M | \nu \rangle
\end{equation}

### 单电子动能的梯度

显然，因为 $\hat t$ 是只关于 $\boldsymbol{r}$ 的参数，因此其算符本身对 $\boldsymbol{A}$ 的导数不作贡献．因此，单电子动能的梯度很容易地表示为

\begin{equation}
t_{\mu \nu}^\boldsymbol{A} = \langle \nabla_\boldsymbol{A} \mu | \hat t | \nu \rangle + \langle \mu | \hat t | \nabla_\boldsymbol{A} \nu \rangle
\end{equation}

我们现在分析项 $\langle \nabla_\boldsymbol{A} \mu | \hat t | \nu \rangle$．我们首先断定，如果 $\mu(\boldsymbol{r} - \boldsymbol{R}_\mu)$ 如果不是 $A$ 原子的原子轨道，即 $\boldsymbol{R}_\mu$ 从定义上不等价于 $\boldsymbol{A}$ (而不是 $\boldsymbol{M}$ 的值不等于 $\boldsymbol{A}$)，那么 $\nabla_\boldsymbol{A} \mu(\boldsymbol{r} - \boldsymbol{R}_\mu) = 0$，因为 $\mu(\boldsymbol{r} - \boldsymbol{R}_\mu)$ 此时与 $\boldsymbol{A}$ 无关．因此，能产生非零贡献的 $\langle \nabla_\boldsymbol{A} \mu | \hat t | \nu \rangle$ 中，$\mu$ 必然是 $A$ 原子的原子轨道，即可以表示为 $\mu(\boldsymbol{r} - \boldsymbol{A})$．

由于一般来说，没有处理对原子核坐标的梯度的积分引擎，我们需要将对 $\boldsymbol{A}$ 的梯度转化为对 $\boldsymbol{r}$ 的梯度．这可以通过坐标变换得到：如果重新定义坐标原点，它在旧坐标系中的点 $\boldsymbol{A}$，并令新坐标系的变元 $\boldsymbol{r}' = \boldsymbol{r} - \boldsymbol{A}$．同时，如果我们对原子核 $A$ 进行扰动，由于电子坐标 $\boldsymbol{r}$ 并没有改变，因此 $\nabla_\boldsymbol{A} \cdot \boldsymbol{r}' = -1$．

根据连续偏导原则，我们知道

\begin{equation}
\nabla_\boldsymbol{A} = (\nabla_\boldsymbol{A} \cdot \boldsymbol{r}') \nabla_{\boldsymbol{r}'} = - \nabla_{\boldsymbol{r}'}
\end{equation}

由此，我们可以立即推断得

\begin{equation}
\langle \nabla_\boldsymbol{A} \mu | \hat t | \nu \rangle = - \langle \nabla_{\boldsymbol{r}'} \mu | \hat t | \nu \rangle \cdot \delta(\mu \in A)
\end{equation}

最后，若动能算符本身的变元平移，也不影响其最终值；因此，上式的值与 $A$ 原子的坐标实无关系．在程序实现中，我们需要处理分量下的积分值

\begin{equation}
\langle \partial_{A_t} \mu | \hat t | \nu \rangle = - \langle \partial_{r_t} \mu | \hat t | \nu \rangle \cdot \delta (\mu \in A)
\end{equation}

这里存在有容易引起混淆的地方．等是右边是关于 $t, \mu, \nu$ 的三角标张量，于是似乎单电子动能梯度与原子核无关．事实上，电子积分引擎确实会给出三角标的积分结果：

In [ ]:
(- mol.intor("int1e_ipkin")).shape

但实际上，上面的公式推导，特别是坐标变换，只有在处于原子 $A$ 下才成立，因此等式左边在 $\mu$ 不在原子 $A$ 上时没有意义．下面的代码中，我们会将没有意义的部分设为零值，引入 `[:, p0:p1, :]` 的截取在 $A$ 原子上的原子轨道 $\mu$，得到如下的四脚标的 $A, t, \mu, \nu$ 的张量 $\langle \partial_{A_t} \mu | \hat t | \nu \rangle$：

In [ ]:
tensor_grad_kin = np.zeros((mol.natm, 3, nao, nao))
for atm_id in range(mol.natm):
    _, _, p0, p1 = mol.aoslice_by_atom()[atm_id]
    tensor_grad_kin[atm_id, :, p0:p1, :] = - mol.intor("int1e_ipkin")[:, p0:p1, :]

需要指出，这并不是一种高效的储存方案，因为零值过多．在最后我们会提供一种零值较少的解决方案．

我们现在已经可以计算单电子动能对梯度的贡献大小了．事实上，$\langle \nabla_\boldsymbol{A} \mu | \hat t | \nu \rangle$ 与 $\langle \mu | \hat t | \nabla_\boldsymbol{A} | \nu \rangle$ 之间的关系是 $\mu$ 与 $\nu$ 的转置，利用上面得到的张量转置即可得到．因此，

\begin{align}
2 t^{A_t} = 2 t_{ii}^{A_t} &= 2 C_{\mu i} C_{\nu i} (\langle \partial_{A_t} \mu | \hat t | \nu \rangle + \langle \mu | \hat t | \partial_{A_t} \nu \rangle)\\
&= D_{\mu \nu} (\langle \partial_{r_t} \mu | \hat t | \nu \rangle \cdot \delta(\mu \in A) + \langle \mu | \hat t | \partial_{r_t} \nu \rangle \cdot \delta(\nu \in A)) 
\end{align}

上面之所以在等式左右乘上两倍，可以看作是因为 [核坐标梯度公式](#核坐标梯度公式) 中的 $h_{ii}^{A_t}$ 本身带有两倍的贡献，也可以看作是 RHF 的自旋轨道代入时自然需要引出的两倍．

In [ ]:
grad_kin = np.einsum("uv, Atuv -> At", D, tensor_grad_kin + tensor_grad_kin.transpose(0, 1, 3, 2))
grad_kin

### 势能积分的补充说明

在继续梯度代码实现前，我们先对势能积分的另一种导出方式进行说明．我们在 HF 方法下 Fock 矩阵的分解的说明中，使用下面的代码输出势能积分 $v^\mathrm{nuc}_{\mu \nu} = \langle \mu | \hat v_M | \nu \rangle$：

In [ ]:
Vnuc = mol.intor("int1e_nuc")

事实上，上面的积分还可以通过另一种方式导出：

In [ ]:
Vnuc_rinv_method = np.zeros((nao, nao))
for atm_id in range(mol.natm):
    with mol.with_rinv_as_nucleus(atm_id):
        Vnuc_rinv_method += mol.intor("int1e_rinv") * -mol.atom_charge(atm_id)
np.allclose(Vnuc_rinv_method, Vnuc)

这里有必要显式地给出 $\mu$ 与 $\nu$ 的 GTO 函数的变量 $\mu (\boldsymbol{r} - \boldsymbol{R}_\mu)$ 与 $\nu (\boldsymbol{r} - \boldsymbol{R}_\nu)$．那么，对于每一个原子 $M$ 分别进行坐标平移后再求和，就可以得到下面的等式：

\begin{equation}
\sum_{M} \langle \mu(\boldsymbol{r} - \boldsymbol{R}_\mu) | \frac{-Z_M}{| \boldsymbol{r} - \boldsymbol{M} |} | \nu (\boldsymbol{r} - \boldsymbol{R}_\nu) \rangle =
\sum_{M} \langle \mu(\boldsymbol{r} + \boldsymbol{M} - \boldsymbol{R}_\mu) | \frac{-Z_M}{| \boldsymbol{r} |} | \nu (\boldsymbol{r} + \boldsymbol{M} - \boldsymbol{R}_\nu) \rangle
\end{equation}

或者用也许不太适合的 Einstein Summation Convention，可以记为

\begin{equation}
v^\mathrm{nuc}_{\mu \nu} = \langle \mu(\boldsymbol{r} + \boldsymbol{M} - \boldsymbol{R}_\mu) | \frac{-Z_M}{| \boldsymbol{r} |} | \nu (\boldsymbol{r} + \boldsymbol{M} - \boldsymbol{R}_\nu) \rangle
\end{equation}

这就将所有原先算符中带的原子核坐标移步到左矢与右矢了，因此这可以看作关于 $1 / |\boldsymbol{r}|$ 的积分．处理 $\mu(\boldsymbol{r} + \boldsymbol{R}_\mu)$ 到 $\mu(\boldsymbol{r} + \boldsymbol{M} - \boldsymbol{R}_\mu)$ 的坐标变换的方法很简单：后者坐标原点即为前者的 $\boldsymbol{M}$，那么 $\mu(\boldsymbol{r} + \boldsymbol{M} - \boldsymbol{R}_\mu)$ 代表的意义即是取 $\boldsymbol{M}$ 为新的坐标原点后的轨道函数．

在 PySCF 中，暂时地移动分子的坐标可以使用 `gto.mole.Mole.with_rinv_as_nucleus` 方法．举个例子来说，我们可以打出当前分子的坐标：

In [ ]:
mol.atom_coords()

下面我们会在 `with` 语句内临时地在求电子积分会传入第二个原子，即第一个氢原子的坐标作为积分原点；而离开 `with` 后，传入的积分原点就是零坐标了．具体的积分传参过程我也不甚了解，但相信应当是通过类 `mole` 内保护变量 `_env` 进行的，其第 4 到 7 个参数是传入的积分原点：

In [ ]:
with mol.with_rinv_as_nucleus(1):
    print("Inside with")
    print(mol._env[4:7])
    tmp1 = mol.intor("int1e_rinv")
print("Outside with")
print(mol._env[4:7])
tmp2 = mol.intor("int1e_rinv")
np.allclose(tmp1, tmp2, atol=1e-4)

### 单电子势能左矢梯度

$\nabla_\boldsymbol{A} \langle \mu | \hat v_M | \nu \rangle$ 可以拆成三部分考虑，分别是左矢梯度 $\langle \nabla_\boldsymbol{A} \mu | \hat v_M | \nu \rangle$、算符梯度 $\langle \mu | \nabla_\boldsymbol{A} \hat v_M | \nu \rangle$ 与右矢梯度 $\langle \mu | \hat v_M | \nabla_\boldsymbol{A} \nu \rangle$．右矢与左矢梯度之间可以看作是矩阵的转置关系，因此实际上我们只需要考虑两种情况．这部分中，我们会考虑左矢梯度部分 $\langle \nabla_\boldsymbol{A} \mu | \hat v_M | \nu \rangle$．

仿照上面的坐标变换的说明，我们很容易地给出 (下式对 $M$ 求和)

\begin{align}
&\quad \langle \nabla_\boldsymbol{A} \mu (\boldsymbol{r} - \boldsymbol{R}_\mu) | \frac{- Z_M}{| \boldsymbol{r} - \boldsymbol{M} |} | \nu (\boldsymbol{r} - \boldsymbol{R}_\nu) \rangle \\
&= \langle \nabla_\boldsymbol{A} \mu (\boldsymbol{r} - \boldsymbol{A}) | \frac{- Z_M}{| \boldsymbol{r} - \boldsymbol{M} |} | \nu (\boldsymbol{r} - \boldsymbol{R}_\nu) \rangle \cdot \delta(\mu \in A) \\
&= - \langle \nabla_\boldsymbol{r} \mu (\boldsymbol{r} - \boldsymbol{A}) | \frac{- Z_M}{| \boldsymbol{r} - \boldsymbol{M} |} | \nu (\boldsymbol{r} - \boldsymbol{R}_\nu) \rangle \cdot \delta(\mu \in A) \\
&= - \langle \nabla_\boldsymbol{r} \mu (\boldsymbol{r} + \boldsymbol{M} - \boldsymbol{A}) | \frac{- Z_M}{| \boldsymbol{r} |} | \nu (\boldsymbol{r} + \boldsymbol{M} - \boldsymbol{R}_\nu) \rangle \cdot \delta(\mu \in A)
\end{align}

其中，上式的第一个等号表示只有当 $\mu$ 是 $A$ 的原子轨道时，上式才有意义；第二个等号利用 [上面](#单电子动能的梯度) 提到的 $\nabla_\boldsymbol{r} = \nabla_\boldsymbol{A} \cdot \boldsymbol{r} \nabla_\boldsymbol{A} = - \nabla_\boldsymbol{A}$ 关系；第三个等号则是利用 [坐标平移变换](#势能积分的补充说明)，分别将每一项中 $M$ 原子作为原点求取积分后加和．

可以看到，这与方才求取势能积分时所采用的公式是及其类似的；只是需要注意到上式中 $A$ 与 $\mu$ 存在的隐含关系．我们记上式的结果为四脚标张量 $^\mathrm{1} v_{\mu \nu}^{A_t}$；左上标 1 并没有特殊意义，只是与下面提到的 [单电子势能算符梯度](#单电子势能算符梯度) 区分开来而已．代码实现如下：

In [ ]:
tensor_grad_v1 = np.zeros((mol.natm, 3, nao, nao))
for atm_id in range(mol.natm):
    _, _, p0, p1 = mol.aoslice_by_atom()[atm_id]
    for origin_id in range(mol.natm):
        with mol.with_rinv_as_nucleus(origin_id):
            tensor_grad_v1[atm_id, :, p0:p1, :] += mol.intor("int1e_iprinv")[:, p0:p1, :] * mol.atom_charge(origin_id)

获得该张量后，我们可以立即得到其对核哈密顿的梯度贡献：

\begin{align}
2 {}^1 v^{A_t} = 2 {}^1 v_{ii}^{A_t} 
&= D_{\mu \nu} \bigg( \langle \partial_{r_t} \mu (\boldsymbol{r} + \boldsymbol{M} - \boldsymbol{A}) | \frac{-Z_M}{|\boldsymbol{r}|} | \nu (\boldsymbol{r} + \boldsymbol{M} - \boldsymbol{R}_\nu) \rangle \cdot \delta(\mu \in A) \\
&\quad + \langle \mu (\boldsymbol{r} + \boldsymbol{M} - \boldsymbol{R}_\mu) | \frac{-Z_M}{|\boldsymbol{r}|} | \partial_{r_t} \nu (\boldsymbol{r} + \boldsymbol{M} - \boldsymbol{A}) \rangle \cdot \delta(\nu \in A) \bigg)
\end{align}

In [ ]:
grad_v1 = np.einsum("uv, Atuv -> At", D, tensor_grad_v1 + tensor_grad_v1.transpose(0, 1, 3, 2))
grad_v1

这里指出，尽管由于存在两项颠倒 $\mu$ 与 $\nu$ 的位置的项，因此需要引入张量转置；但实际上，由于 $\mu$ 与 $\nu$ 在计算梯度时被求和，因此实际上未必需要张量转置，简单地乘以二也是可以的．下面代码验证这个结论．

In [ ]:
np.allclose(np.einsum("uv, Atuv -> At", D, tensor_grad_v1 + tensor_grad_v1.transpose(0, 1, 3, 2)),
    np.einsum("uv, Atuv -> At", D, tensor_grad_v1 * 2))

不过上面的表达式在 PySCF ([libcint 引擎](https://github.com/sunqm/libcint)) 中有更简单的实现．注意到上面的连等式的第二个等式右方的表达式还可以写为

\begin{equation}
- \langle \nabla_\boldsymbol{r} \mu (\boldsymbol{r} - \boldsymbol{A}) | \frac{- Z_M}{| \boldsymbol{r} - \boldsymbol{M} |} | \nu (\boldsymbol{r} - \boldsymbol{R}_\nu) \rangle \cdot \delta(\mu \in A)
= - \langle \nabla_\boldsymbol{r} \mu (\boldsymbol{r} - \boldsymbol{A}) | \hat v_\mathrm{nuc} | \nu (\boldsymbol{r} - \boldsymbol{R}_\nu) \rangle \cdot \delta(\mu \in A)
\end{equation}

因此，不一定需要真的使用处理 $1/|\boldsymbol{r}|$ 的积分方法 `int1e_iprinv`，而可以直接使用处理电子与原子核算符 $\hat v_\mathrm{nuc}$ 的积分方法 `int1e_ipnuc` 即可实现张量的导出．

In [ ]:
tensor_grad_v1_usenuc = np.zeros((mol.natm, 3, nao, nao))
for atm_id in range(mol.natm):
    _, _, p0, p1 = mol.aoslice_by_atom()[atm_id]
    tensor_grad_v1_usenuc[atm_id, :, p0:p1, :] -= mol.intor("int1e_ipnuc")[:, p0:p1, :]
np.allclose(tensor_grad_v1_usenuc, tensor_grad_v1)

不过将 $\hat v_\mathrm{nuc}$ 不看作一个整体，而拆分为对于 $\hat v_M$ 的每一个分项的计算这件事仍然是有意义的．我们将在下一部分中立即看到其应用．

### 单电子势能算符梯度

现在我们处理 $\langle \mu | \nabla_\boldsymbol{A} \hat v_M | \nu \rangle$ 的结果．我们下面会简单地说明，这种对算符的偏导可以化为对左右矢的偏导，从而避免直接对算符进行偏导计算．

首先，由于 $\hat v_M$ 中，只有当 $M$ 等价于 $A$ 时，其与 $\boldsymbol{A}$ 有关；因此，

\begin{equation}
\langle \mu | \nabla_\boldsymbol{A} \hat v_M | \nu \rangle = \langle \mu | \nabla_\boldsymbol{A} \hat v_A | \nu \rangle = \langle \mu | \nabla_\boldsymbol{A} \frac{-Z_A}{| \boldsymbol{r} - \boldsymbol{A} |} | \nu \rangle
\end{equation}

再利用 $\nabla_\boldsymbol{r} = \nabla_\boldsymbol{A} \cdot \boldsymbol{r} \nabla_\boldsymbol{A} = - \nabla_\boldsymbol{A}$ 关系式，可以得到

\begin{equation}
\langle \mu | \nabla_\boldsymbol{A} \hat v_M | \nu \rangle = - \langle \mu | \nabla_\boldsymbol{r} \frac{-Z_A}{| \boldsymbol{r} - \boldsymbol{A} |} | \nu \rangle
\end{equation}

注意到上一步的转换中，我们没有强加 $\delta(\mu \in A)$ 的要求．这是因为在这里，被求导对象并非是原子轨道，而是算符．因此，只有算符是不是在原子 $A$ 上，而没有轨道是不是在 $A$ 上的限制了．

如果我们现在转换坐标原点到原子核 $A$ 的中心，并记 $\mu', \nu'$ 为转换后的原子轨道；我们可以将上式结果记作

\begin{equation}
\langle \mu | \nabla_\boldsymbol{A} \hat v_M | \nu \rangle = - \langle \mu' | \nabla_\boldsymbol{r} \frac{-Z_A}{| \boldsymbol{r} |} | \nu' \rangle
\end{equation}

显然，$-Z_A / | \boldsymbol{r} |$ 可以当做一个简单函数对待；如果我们假定原子轨道表达式是实数，那么活用 Dirac 记号，我们可以将上式写作

\begin{equation}
\langle \mu | \nabla_\boldsymbol{A} \hat v_M | \nu \rangle = - \langle \nabla_\boldsymbol{r} \frac{-Z_A}{| \boldsymbol{r} |} | \mu' \nu' \rangle
\end{equation}

如果我们接受 $\nabla_\boldsymbol{r}$ 是反厄米算符这个事实，那么通过上式，我们可以使用我们熟悉的方式求取积分：

\begin{equation}
\langle \mu | \nabla_\boldsymbol{A} \hat v_M | \nu \rangle = \langle \frac{-Z_A}{| \boldsymbol{r} |} | \nabla_\boldsymbol{r} (\mu' \nu') \rangle
= \langle \nabla_\boldsymbol{r} \mu' | \frac{-Z_A}{| \boldsymbol{r} |} | \nu' \rangle + \langle \mu' | \frac{-Z_A}{| \boldsymbol{r} |} | \nabla_\boldsymbol{r} \nu' \rangle
\end{equation}

从而单电子势能算符梯度对能量梯度的贡献表示为

\begin{equation}
2 {}^2 v^{A_t} = 2 {}^2 v_{ii}^{A_t} = D_{\mu \nu} \langle \mu | \nabla_\boldsymbol{A} \hat v_M | \nu \rangle
= D_{\mu \nu} \left( \langle \nabla_\boldsymbol{r} \mu' | \frac{-Z_A}{| \boldsymbol{r} |} | \nu' \rangle + \langle \mu' | \frac{-Z_A}{| \boldsymbol{r} |} | \nabla_\boldsymbol{r} \nu' \rangle \right)
\end{equation}

In [ ]:
tensor_grad_v2 = np.zeros((mol.natm, 3, nao, nao))
for atm_id in range(mol.natm):
    _, _, p0, p1 = mol.aoslice_by_atom()[atm_id]
    with mol.with_rinv_as_nucleus(atm_id):
        tensor_grad_v2[atm_id, :] -= mol.intor("int1e_iprinv") * mol.atom_charge(atm_id)
grad_v2 = np.einsum("uv, Atuv -> At", D, tensor_grad_v2 + tensor_grad_v2.transpose(0, 1, 3, 2))
grad_v2

同上地，代码中未必要显式地进行转置：

In [ ]:
np.allclose(np.einsum("uv, Atuv -> At", D, tensor_grad_v2 + tensor_grad_v2.transpose(0, 1, 3, 2)),
    np.einsum("uv, Atuv -> At", D, tensor_grad_v2 * 2))

### $\nabla_\boldsymbol{r}$ 作为反厄米算符

我们指出，$\nabla_\boldsymbol{r}$ 是一个反厄米算符．事实上，刚才在程序中一直出现的 `int1e_ipkin` 的 $\langle \nabla_{\boldsymbol{r}} \mu | \hat t | \nu \rangle$，以及将来出现的 `int1e_ipovlp` 的 $\langle \nabla_{\boldsymbol{r}} \mu | \nu \rangle$ 输出的矩阵在每一个 $r_t$ 分量上都是反厄米矩阵 (实数下是反对称矩阵)．我们可以验证：

In [ ]:
print([ np.allclose(mol.intor("int1e_ipkin")[i], -mol.intor("int1e_ipkin")[i].T) for i in range(3)])

之所以 $\langle \nabla_{\boldsymbol{r}} \mu | \hat t | \nu \rangle = - \langle \mu | \hat t | \nabla_{\boldsymbol{r}} \nu \rangle = - \langle \nabla_{\boldsymbol{r}} \nu | \hat t | \mu \rangle$ 成立，还要考虑到动能算符 $\hat t = - \frac 1 2 \nabla_{\boldsymbol{r}}^2$ 是由 $\nabla_\boldsymbol{r}$ 构成，因此可以与 $\nabla_{\boldsymbol{r}}$ 对易；以及动能算符为厄米算符的性质：

\begin{align}
\langle \nabla_{\boldsymbol{r}} \mu | \hat t \nu \rangle
= - \langle  \mu | \nabla_{\boldsymbol{r}} \hat t \nu \rangle
= - \langle  \mu | \hat t \nabla_{\boldsymbol{r}} \nu \rangle
\end{align}

但同样的式子不能直接套到 `int1e_rinv` 的 $\langle \nabla_\boldsymbol{r} \mu | \frac{1}{| \boldsymbol{r} |} | \nu \rangle$ 中，因为 $\nabla_\boldsymbol{r}$ 与 $\frac{1}{| \boldsymbol{r} |}$ 应当是不能对易的．

In [ ]:
print([ np.allclose(mol.intor("int1e_iprinv")[i], -mol.intor("int1e_iprinv")[i].T) for i in range(3) ])
print([ np.allclose(mol.intor("int1e_iprinv")[i], mol.intor("int1e_iprinv")[i].T) for i in range(3) ])

关于 $\nabla_\boldsymbol{r}$ 的反厄米性质，可以通过下面两种说法说明：第一，如果我们把问题看成一维问题，那么根据分部积分法则，

\begin{equation}
\int u \partial_r v \mathrm{d} r = \int u \mathrm{d} v = (uv)|_{r = -\infty}^{+\infty} - \int v \mathrm{d} u = - \int v \partial_r u \mathrm{d} r
\end{equation}

前提是函数 $uv$ 在无穷远处值为零或相等．那么类比一维问题，三维下 $\nabla_\boldsymbol{r}$ 也具有这种性质，只是证明需要使用广义 Stocks 定理．
第二，由于 $- i \hbar \nabla_\boldsymbol{r}$ 是动量算符；由于动量算符是厄米算符，因此与其有 $\pi/2$ 复平面幅角差的 $\nabla_\boldsymbol{r}$ 为反厄米算符．

### 核哈密顿对梯度的贡献：总结

到现在为止，我们成功获得了动能的、左右矢梯度的势能的、以及势能算符的梯度贡献值．将它们加起来，就可以得到核哈密顿对梯度的贡献值：

\begin{equation}
2 h^{A_t} = 2 h_{ii}^{A_t} = 2 t^{A_t} + 2 {}^1 v^{A_t} + 2 {}^2 v^{A_t}
\end{equation}

In [ ]:
grad_hcore_4idx_tensor = grad_kin + grad_v1 + grad_v2
grad_hcore_4idx_tensor

在前面我们指出，我们为了计算梯度，使用了含有大量零值的四脚标张量；这不是内存高效的方案．我们可以用下面较为紧凑的代码完成同样的工作，并且中途储存与计算的矩阵角标数量不超过三．角标中被削去的维度是原子角标 $A$；对它的索引从上面几部分中使用张量角标换成了循环．

In [ ]:
int1e_ipkin = mol.intor("int1e_ipkin")
int1e_ipnuc = mol.intor("int1e_ipnuc")

grad_hcore = np.zeros((mol.natm, 3))
for atm_id in range(mol.natm):
    tensor_grad_hcore = np.zeros((3, nao, nao))
    _, _, p0, p1 = mol.aoslice_by_atom()[atm_id]
    tensor_grad_hcore[:, p0:p1, :] -= int1e_ipkin[:, p0:p1, :]
    tensor_grad_hcore[:, p0:p1, :] -= int1e_ipnuc[:, p0:p1, :]
    with mol.with_rinv_as_nucleus(atm_id):
        tensor_grad_hcore[:] -= mol.intor("int1e_iprinv") * mol.atom_charge(atm_id)
    grad_hcore[atm_id, :] = np.einsum("tuv, uv -> t", tensor_grad_hcore * 2, D)
    # *2 above means `+ tensor_grad_hcore.swapaxes(1, 2)`, or `+ tensor_grad_hcore.transpose(0, 2, 1)`
np.allclose(grad_hcore, grad_hcore_4idx_tensor)

## 重叠积分导数对梯度的贡献

接下来我们分析项 $- 2 S_{ii}^a \varepsilon_i$．与方才的核梯度一样，我们需要将分子轨道角标的重叠积分化为原子轨道角标，并将原子坐标分量的记号 $A_t$ 代入上面的抽象坐标记号 $a$ 中：

\begin{equation}
-2 S_{ii}^{A_t} \varepsilon_i = -2 S_{\mu \nu}^{A_t} C_{\mu i} \varepsilon_i C_{\nu i}
\end{equation}

在这一步，我们指出，上式的 $2 C_{\mu i} \varepsilon_i C_{\nu i}$ 又称为能量加权密度；它相对于一般的 SCF 密度 $D_{\mu \nu} = 2 C_{\mu i} C_{\nu i}$ 中间少去轨道能量．我们可以分别用 `numpy.einsum` 生成，抑或使用 PySCF 自带的工具 `grad.rhf.make_rdm1e` 生成该矩阵．

In [ ]:
np.allclose(
    np.einsum("ui, i, vi -> uv", C, mo_occ * e, C),
    grad.rhf.make_rdm1e(e, C, mo_occ))

下面我们需要生成 $S_{\mu \nu}^{A_t}$．我们仍然需要活用关系式 $\nabla_\boldsymbol{r} = \nabla_\boldsymbol{A} \cdot \boldsymbol{r} \nabla_\boldsymbol{A} = - \nabla_\boldsymbol{A}$，有

\begin{align}
S_{\mu \nu}^{A_t} &= \frac{\partial}{\partial A_t} \langle \mu | \nu \rangle \\
&= \langle \frac{\partial}{\partial A_t} \mu | \nu \rangle + \langle \mu | \frac{\partial}{\partial A_t} \nu \rangle \\
&= - \langle \frac{\partial}{\partial r_t} \mu | \nu \rangle \cdot \delta(\mu \in A) - \langle \frac{\partial}{\partial r_t} \nu | \mu \rangle \cdot \delta(\nu \in A)
\end{align}

这里我们无需考虑坐标原点的重定义问题；重叠积分以及其导数不论坐标原点选在何处，结果都应当相同．而在 PySCF 的积分库中，`int1e_ipovlp` 代表的是重叠积分中左侧原子轨道对电子坐标的导数 $\langle \nabla_{\boldsymbol{r}} \mu | \nu \rangle$．

In [ ]:
grad_ovlp = np.zeros((mol.natm, 3))
for atm_id in range(mol.natm):
    _, _, p0, p1 = mol.aoslice_by_atom()[atm_id]
    grad_ovlp[atm_id] = np.einsum('tuv, ui, i, vi ->t', 
        mol.intor('int1e_ipovlp')[:, p0:p1, :], C[p0:p1, :], e * mo_occ, C) * 2
    # *2 refers to u <-> v, or
    # grad_ovlp[atm_id] += np.einsum('tvu, vi, i, ui ->t', 
    #     mol.intor('int1e_ipovlp')[:, p0:p1, :], C[p0:p1, :], e * mo_occ, C)
grad_ovlp

如果使用能量加权密度矩阵来执行上述过程，则可以写为

In [ ]:
De = grad.rhf.make_rdm1e(e, C, mo_occ)
grad_ovlp_De = np.zeros((mol.natm, 3))
for atm_id in range(mol.natm):
    _, _, p0, p1 = mol.aoslice_by_atom()[atm_id]
    grad_ovlp_De[atm_id] = np.einsum('tuv, uv ->t', 
        mol.intor('int1e_ipovlp')[:, p0:p1, :], De[p0:p1, :]) * 2
np.allclose(grad_ovlp, grad_ovlp_De)

## 双电子积分导数对梯度的贡献

在一阶电子态梯度计算的最后，我们考虑双电子积分 $2 (ii|jj)^a - (ij|ij)^a$ 的结果．将原子坐标分量的记号 $A_t$ 代入 $2 (ii|jj)^a$ 的抽象坐标记号 $a$ 中，并作展开：

\begin{align}
2 (ii|jj)^a &= \frac{1}{2} D_{\mu \nu} D_{\kappa \tau} (\mu \nu | \kappa \tau)
= \frac{1}{2} D_{\mu \nu} D_{\kappa \tau} \left[
( \frac{\partial \mu}{\partial A_t} \nu | \kappa \tau )
+ ( \mu \frac{\partial \nu}{\partial A_t} | \kappa \tau )
+ ( \mu \nu | \frac{\partial \kappa}{\partial A_t} \tau )
+ ( \mu \nu | \kappa \frac{\partial \tau}{\partial A_t} )
\right]
\end{align}

我们先取第一项．如果我们定义 $J^a_{\mu \nu}[\mathbf{D}] =  D_{\kappa \lambda} (\partial_a \mu \nu | \kappa \tau )$，那么第一项可以表示为

\begin{equation}
\frac{1}{2} D_{\mu \nu} D_{\kappa \tau} ( \frac{\partial \mu}{\partial A_t} \nu | \kappa \tau ) = D_{\mu \nu} J^{A_t}_{\mu \nu}[\mathbf{D}] = - D_{\mu \nu} J^{r_t}_{\mu \nu}[\mathbf{D}] \cdot \delta(\mu \in A)
\end{equation}

对于交换积分导数 $K^a_{\mu \nu}[\mathbf{D}] = D_{\kappa \lambda} (\partial_a \mu \kappa | \nu \tau )$ 也是类似的．需要指出，上述定义的 $J^a_{\mu \nu}[\mathbf{D}]$ 并非是对称的．在 PySCF 中，有 `grad.rhf.get_jk` 函数同时生成 $J^{r_t}_{\mu \nu}[\mathbf{D}]$ 与 $K^{r_t}_{\mu \nu}[\mathbf{D}]$；我们也可以用 PySCF 的积分函数生成这两个矩阵：

In [ ]:
J_1p, K_1p = grad.rhf.get_jk(mol, D)

In [ ]:
np.allclose(-np.einsum("tuvkl, kl -> tuv", mol.intor('int2e_ip1'), D), J_1p)

In [ ]:
np.allclose(-np.einsum("tukvl, kl -> tuv", mol.intor('int2e_ip1'), D), K_1p)

In [ ]:
print([ np.allclose(J_1p[i], J_1p[i].T) for i in range(3) ])
print([ np.allclose(J_1p[i], -J_1p[i].T) for i in range(3) ])

对于库伦积分的导数，四项的值可以简单地化为两项，因为 $\mu, \nu$ 与 $\kappa, \lambda$ 均是角标，是可以任意交换的；因此

\begin{align}
2 (ii|jj)^{A_t} &= - D_{\mu \nu} J_{\mu \nu}^{r_t} [\textbf{D}] \cdot \delta(\mu \in A) - D_{\mu \nu} J_{\nu \mu}^{r_t} [\textbf{D}] \cdot \delta(\nu \in A) \\
&= - D_{\mu \nu} J_{\mu \nu}^{r_t} [\textbf{D}] \cdot \delta(\mu \in A) - D_{\nu \mu} J_{\mu \nu}^{r_t} [\textbf{D}] \cdot \delta(\mu \in A) \\
&= - 2 D_{\mu \nu} J_{\mu \nu}^{r_t} [\textbf{D}] \cdot \delta(\mu \in A)
\end{align}

上式的最后一个等式利用的是密度矩阵的对称性．对于交换积分是相同的．我们可以将上面的计算过程用代码表示：

In [ ]:
grad_eri = np.zeros((mol.natm, 3))
for atm_id in range(mol.natm):
    _, _, p0, p1 = mol.aoslice_by_atom()[atm_id]
    grad_eri[atm_id] += np.einsum('tuv, uv->t', (J_1p - 0.5 * K_1p)[:, p0:p1, :], D[p0:p1, :]) * 2
grad_eri

## 总一阶核坐标梯度

### 总一阶梯度的结果

我们已经把一阶梯度的三个分项计算完毕了．电子态能量在坐标系下的梯度最终可以写作

In [ ]:
grad_elec = grad_hcore + grad_ovlp + grad_eri
grad_elec

我们可以与 PySCF 给出的结果进行比较：

In [ ]:
np.allclose(grad_elec, scf_grad.grad_elec())

但是需要指出，上述的结果是电子态能量 $E_\mathrm{elec}$ 的一阶梯度．考虑到总能量中还包含原子核排斥能 $E_\mathrm{nuc}$，这部分的梯度也应当算在总能量梯度里．

原子核排斥能可以用下面的记号写出来：

\begin{equation}
E_\mathrm{nuc} = \frac{1}{2} \frac{Z_N Z_M}{|\boldsymbol{N} - \boldsymbol{M}|}
\end{equation}

上式中，$N$ 与 $M$ 不能是相同的原子．对于 $A$ 原子坐标的梯度则可以表示为

\begin{equation}
\nabla_\boldsymbol{A} E_\mathrm{nuc} = - \frac{Z_A Z_M}{|\boldsymbol{A} - \boldsymbol{M}|^3} (\boldsymbol{A} - \boldsymbol{M})
\end{equation}

上式的实现可以简单地用循环实现：

In [ ]:
grad_nuc = np.zeros((mol.natm, 3))
for A in range(mol.natm):
    for M in range(mol.natm):
        if A == M: continue;
        vec = mol.atom_coord(A) - mol.atom_coord(M)
        grad_nuc[A, :] -= mol.atom_charge(A) * mol.atom_charge(M) * vec / np.linalg.norm(vec) ** 3
grad_nuc

我们也可以将上述的结果与 PySCF 给出的结果比较：

In [ ]:
np.allclose(grad_nuc, scf_grad.grad_nuc())

将上述两个梯度矩阵加和，即得到最终的一阶梯度；并与 PySCF 的一阶梯度进行比较：

In [ ]:
grad_total = grad_elec + grad_nuc
grad_total

In [ ]:
np.allclose(grad_total, scf_grad.grad())

### 代码总结

在一阶梯度部分结束之前，我们将上述的代码进行总结．可以看出，尽管一阶梯度的计算稍繁杂，但单是代码上，长度其实不长．

In [ ]:
def my_grad_elec(scf_eng):
    mol = scf_eng.mol
    C = scf_eng.mo_coeff
    D = scf_eng.make_rdm1()
    mo_occ = scf_eng.mo_occ
    e = scf_eng.mo_energy
    De = np.einsum("ui, i, vi -> uv", C, e * mo_occ, C)
    integrals  = - mol.intor("int1e_ipkin") - mol.intor("int1e_ipnuc") \
                 - np.einsum("tuvkl, kl -> tuv", mol.intor('int2e_ip1'), D) \
                 + 0.5 * np.einsum("tukvl, kl -> tuv", mol.intor('int2e_ip1'), D)
    integral_ovlp = mol.intor('int1e_ipovlp')
    grad_elec = np.zeros((mol.natm, 3))
    for atm_id in range(mol.natm):
        _, _, p0, p1 = mol.aoslice_by_atom()[atm_id]
        grad_elec[atm_id, :] += np.einsum("tuv, uv -> t", integrals[:, p0:p1], D[p0:p1]) * 2
        grad_elec[atm_id, :] += np.einsum("tuv, uv -> t", integral_ovlp[:, p0:p1], De[p0:p1]) * 2
        with mol.with_rinv_as_nucleus(atm_id):
            grad_elec[atm_id, :] -= np.einsum("tuv, uv -> t", mol.intor("int1e_iprinv") * mol.atom_charge(atm_id), D) * 2
    return grad_elec

def my_grad_nuc(scf_eng):
    mol = scf_eng.mol
    grad_nuc = np.zeros((mol.natm, 3))
    for A in range(mol.natm):
        for M in range(mol.natm):
            if A == M: continue;
            vec = mol.atom_coord(A) - mol.atom_coord(M)
            grad_nuc[A, :] -= mol.atom_charge(A) * mol.atom_charge(M) * vec / np.linalg.norm(vec) ** 3
    grad_nuc
    return grad_nuc

def my_grad(scf_eng):
    return my_grad_elec(scf_eng) + my_grad_nuc(scf_eng)

## 结构优化

一阶核坐标梯度最常见的应用是在结构优化上．PySCF 的结构优化通过 [berny 库](https://jan.hermann.name/pyberny/) 实现．下面我们会了解，如何通过方才的梯度程序进行最初步的结构优化过程．在此之前，我们有必要确认是否可以执行下述代码：

In [ ]:
from pyscf import geomopt
from berny import Berny, geomlib

如果出错的话，需要先安装 berny 库：

```bash
$ pip install berny
```

为了后面不会输出太多信息，这里静默 PySCF 的大多数输出：

In [ ]:
mol.verbose = 0

### PySCF 自带的结构优化

PySCF 的结构优化代码可以执行如下：

In [ ]:
opt_scf = scf.RHF(mol)
mol_opt_pyscf = geomopt.optimize(opt_scf,
    gradientmax=0.000001, gradientrms=0.000001, stepmax=0.000002, steprms=0.000002)

上面函数的收敛标准采用 Gaussian 的 `Optimize(VeryTight)`；其设定参数的方法并非在 PySCF 中记录，而是在 berny 的 [API 文档](https://jan.hermann.name/pyberny/api.html#berny.berny.defaults) 中．作为结果的 `mol_opt_pyscf` 的类型是 `pyscf.gto.mole.Mole`，它包含了优化完毕的分子构型：

In [ ]:
mol_opt_pyscf.atom_coords()

上述构型的单位是 Bohr．我们可以将其转化为 Z-matrix，与 [Gaussian 程序的输出](include/HF-grad.gjf) 进行比对．需要注意下面 Z-matrix 中键长的单位也是 Bohr，与 Gaussian 输出的 Angstrom 相差大约 $0.529177$ 倍．

In [ ]:
print(gto.mole.cart2zmat(mol_opt_pyscf.atom_coords()))

### PySCF 类继承结构优化

刚刚我们代入结构优化的类是

In [ ]:
opt_scf.__class__

其在函数 `geomopt.optimize` 中通过传入下述值来计算梯度：

In [ ]:
opt_scf.nuc_grad_method().__class__

如果我们想要利用 PySCF 的结构优化函数，同时使用自己编写的梯度计算函数，最方便的方法是对上述类与函数分别进行继承于重载．下面就是继承与重载的方案：

In [ ]:
class MyGradient(grad.rhf.Gradients):
    
    def __init__(self, mf):
        grad.rhf.Gradients.__init__(self, mf)
        
    def grad_elec(self, mo_energy=None, mo_coeff=None, mo_occ=None, atmlst=None):
        return my_grad_elec(self.base)
    
    def grad_nuc(self, mol=None, atmlst=None):
        return my_grad_nuc(self.base)
    
class MySCF(scf.hf.RHF):
    
    def __init__(self, mf):
        scf.hf.RHF.__init__(self, mf)
        
    def nuc_grad_method(self):
        return MyGradient(self)

In [ ]:
tmp_grad = MyGradient(scf_eng)
tmp_grad.kernel()

上述的输出看起来好像与使用 `grad.rhf.Gradients` 类的输出与结果都相同，但实际上调用的是我们利用 `scf.hf.RHF` 的 SCF 结果与积分库算写的 `my_grad_elec` 与 `my_grad_nuc` 函数进行计算，并不真正使用原本的 `grad.rhf.Gradients` 的成员函数．可以在 `my_grad_elec` 或 `my_grad_nuc` 增加调试输出，或者写错误的函数输出，就能看出差别了．

借用上述定义的类，我们可以利用 PySCF 所定义的函数进行梯度计算，并给出最终分子构型的 Z-Matrix：

In [ ]:
opt_scf = MySCF(mol)
mol_opt_inhert = geomopt.optimize(opt_scf,
    gradientmax=0.000001, gradientrms=0.000001, stepmax=0.000002, steprms=0.000002)

In [ ]:
print(gto.mole.cart2zmat(mol_opt_inhert.atom_coords()))

### 直接使用 berny 库的结构优化

在一些情况下，我们也许会需要更为自由可控的结构优化程序．若要直接使用 berny 库优化，尽管可能会增加少许代码复杂度，但不用受制于 PySCF 的结构优化工具；也不必一定要对现有类与函数进行继承于重载，或者为了结构优化而新建一个类．

berny 库在结构优化过程中，中间媒介是 `berny.geomlib.Geometry` 类；优化的集合结构信息储存在该类中．因此，需要构建从 PySCF 的 `pyscf.gto.mole.Mole` 到其的相互转换．这可以自己构建，也可以利用 PySCF 的现有函数进行．代码如下：

In [ ]:
def mol_to_geom(mol):
    # almost idential to PySCF: `geomopt.berny_solver.to_berny_geom(mol)`
    species = [ mol.atom_symbol(i) for i in range(mol.natm) ]
    coords = mol.atom_coords() * lib.param.BOHR
    geom = geomlib.Geometry(species, coords)
    return geom

def geom_to_mol(mol, geom):
    # almost idential to PySCF: `mol_ret = mol.copy().set_geom_(geomopt.berny_solver._geom_to_atom(mol, geom), unit='Bohr')`
    mol_ret = mol.copy()
    mol_ret.set_geom_(geom.coords)
    return mol_ret

同时，我们也需要构建 berny 库在优化中所必须的能量与梯度信息：

In [ ]:
def solver(mol, geom):
    # self-defined function
    scf_eng = scf.RHF(mol)
    energy = scf_eng.kernel()
    gradients = my_grad_elec(scf_eng) + my_grad_nuc(scf_eng)
    return energy, gradients

随后，只要再提供初猜，优化程序就可以进行了．

In [ ]:
optimizer = Berny(mol_to_geom(mol), verbosity=-2,
    gradientmax=0.000001, gradientrms=0.000001, stepmax=0.000002, steprms=0.000002)

for geom in optimizer:
    mol_opt = geom_to_mol(mol, geom)
    energy, gradients = solver(mol_opt, geom)
    optimizer.send((energy, gradients))
    
mol_opt_pyberny = geom_to_mol(mol, geom)

In [ ]:
print(gto.mole.cart2zmat(mol_opt_pyberny.atom_coords()))

尽管结果与上述的计算稍有不同，但可以通过提高输出级别，可以看出这次优化过程的能量、坐标、梯度与优化次数均与 PySCF 自带的结构优化过程相近．